# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint 

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
part_one = "../weatherpy/Resources/citiesall.csv"
part_one = pd.read_csv(part_one)
part_one

,City,Country,Temperature(F),Latitude,Longitude,Humidity(%),Cloudiness(%),Windspeed(mph)
0,hilo,US,62.11,19.73,-155.09,64.0,90.0,8.05
1,taolanaro,AU,59.07,-42.78,147.06,67.0,40.0,9.17
2,new norfolk,BR,89.82,-2.89,-40.12,54.0,32.0,11.43
3,acarau,KZ,-4.97,52.83,70.78,93.0,100.0,17.05
4,stepnyak,ZA,74.41,-33.93,18.42,53.0,0.0,14.99
...,...,...,...,...,...,...,...,...
567,port macquarie,LY,64.44,29.03,21.55,38.0,0.0,10.96
568,bose,US,39.63,40.77,-72.85,56.0,90.0,13.87
569,coahuayana,QA,64.90,25.29,51.53,68.0,0.0,5.82
570,pisco,BR,87.01,-22.12,-45.05,53.0,33.0,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = part_one[["Latitude", "Longitude"]]
humi = part_one["Humidity(%)"].astype(float)


In [5]:
part_one = part_one.dropna()


In [6]:
fig = gmaps.figure(center=(20.0,0.0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humi, dissipating = False, max_intensity=50, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#ideal vacation conditions:1. max temperature 75<x<90 2. Cloudiness <20% 3. Humidity <40%


In [8]:
ideal_df=part_one[(part_one['Temperature(F)']>75) & (part_one['Temperature(F)']<90) & (part_one['Humidity(%)']<40) & (part_one['Cloudiness(%)']<20)]
ideal_df.reset_index()

,index,City,Country,Temperature(F),Latitude,Longitude,Humidity(%),Cloudiness(%),Windspeed(mph)
0,363,paamiut,AR,88.66,-35.73,-61.75,17.0,0.0,17.11
1,376,esil,AR,88.14,-38.37,-60.28,19.0,0.0,23.44
2,463,hobyo,BW,88.23,-20.18,25.23,36.0,5.0,7.87
3,501,akureyri,UY,80.06,-34.48,-54.33,26.0,0.0,9.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df=pd.DataFrame(ideal_df,columns=['City','Country','Latitude','Longitude'])
hotel_df['Hotel Name']= ""
hotel_df.dropna()
hotel_df.reset_index()



,index,City,Country,Latitude,Longitude,Hotel Name
0,363,paamiut,AR,-35.73,-61.75,
1,376,esil,AR,-38.37,-60.28,
2,463,hobyo,BW,-20.18,25.23,
3,501,akureyri,UY,-34.48,-54.33,


In [17]:
hotel_name=[]
target_type = "lodging"
radius = 5000

params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}
   # Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
      
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json=response.json()
    #pprint(response_json)
    try: 
        hotel_name.append(response_json['results'][0]['name'])
    
        hotel_df.loc[index, "Hotel Name"] = response_json["results"][0]["name"]
    
        print(f"The closest hotel to {row[0]} is {response_json['results'][0]['name']}")  
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
The closest hotel to esil is Habitacion Triple con baño y entrada privada
The closest hotel to hobyo is Gweta Lodge
The closest hotel to akureyri is Beleza Pura B&B


In [18]:
hotel_df
hotel_df_new=hotel_df.dropna()
hotel_df_new.reset_index()

,index,City,Country,Latitude,Longitude,Hotel Name
0,363,paamiut,AR,-35.73,-61.75,
1,376,esil,AR,-38.37,-60.28,Habitacion Triple con baño y entrada privada
2,463,hobyo,BW,-20.18,25.23,Gweta Lodge
3,501,akureyri,UY,-34.48,-54.33,Beleza Pura B&B


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_new.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


fig

        


Figure(layout=FigureLayout(height='420px'))

In [21]:
fig = gmaps.figure(center=(20.0,0.0),zoom_level=1)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))